## Import libraries and data


In [1]:
import pandas
from surprise import Dataset, Reader, SVD, model_selection

ModuleNotFoundError: No module named 'surprise'

In [ ]:
movies = pandas.read_csv('data/movies.csv')
credits = pandas.read_csv('data/credits.csv')
ratings = pandas.read_csv("ratings.csv")[["userId", "movieId", "rating"]]

## Create the dataset


Collaborative-Based filtering


In [ ]:
reader = Reader(rating_scale=(1, 5))
dataset = Dataset.load_from_df(ratings, reader)
dataset

## Build the trainset


In [ ]:
trainset = dataset.build_full_trainset()

In [ ]:
list(trainset.all_ratings())

## Train the Model


In [ ]:
svd = SVD()

In [ ]:
svd.fit(trainset)

In [ ]:
svd.predict(15, 1956)

In [ ]:
model_selection.cross_validate(svd, dataset, measures=["RMSE", "MAE"])

In [ ]:
def get_top_n(predictions, n=10):
    top_n = sorted(predictions, key=lambda x: x.est, reverse=True)[:n]

    top_n_iids = [x.iid for x in top_n]

    top_n_movies = []
    found_movies = 0
    for iid in top_n_iids:
        movie_info = movies[movies["id"] == iid].merge(
            ratings, left_on="id", right_on="movieId")
        if not movie_info.empty:
            movie_title = movie_info["title"].iloc[0]
            top_n_movies.append(movie_title)
            found_movies += 1
        else:
            print(f"Movie with ID {iid} not found.")

        if found_movies == n:
            break  # Found enough movies, exit the loop

    return top_n_movies

In [ ]:

get_top_n(svd.test(trainset.build_testset()))